In [1]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow.keras.backend as K
import sys
import datetime
from tensorflow.python.client import device_lib

from sificc_lib import AI, utils, Event, Simulation, root_files, DataModel
np.set_printoptions(precision=2, linewidth=85, suppress=True)

%matplotlib inline

In [2]:
# Check used versions
now = datetime.datetime.now()
# os.environ["CUDA_VISIBLE_DEVICES"]="1" #on doubleCuda

print("Date-time:          ", now) 
print("Tf version:         ", keras.__version__)
print("Numpy version:      ", np.__version__)
print("Python              ", sys.executable)
print("Python version      ", sys.version)
print("GPU devices:        ", device_lib.list_local_devices())

Date-time:           2021-08-09 09:42:47.424109
Tf version:          2.3.0-tf
Numpy version:       1.19.1
Python               /net/software_g4rt/anaconda3/envs/condaenv-sificcnn/bin/python
Python version       3.7.9 (default, Aug 31 2020, 12:42:55) 
[GCC 7.3.0]
GPU devices:         [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6512054234657803271
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10617392668255476276
physical_device_desc: "device: XLA_CPU device"
]


In [3]:
# Model to test with low statistics data set

model_name = 'model-2b-lsr-example-low'

shuffle_clusters = False

# load the training data
data = DataModel('data-low.npz', 
                 batch_size = 128, validation_percent = .05, test_percent = .1)
                 
# append an extra dimention to the features since we are using convolutional layers
data.append_dim = True

# create an AI instance
ai = AI(data, model_name)

ai.weight_type = 2
ai.weight_pos_x = 2.5
ai.weight_pos_y = 1
ai.weight_pos_z = 2
ai.weight_energy = 1.5
ai.weight_e_cluster = 1
ai.weight_p_cluster = 1

# randomly shuffle the training data
#np.random.seed(888)
#ai.data.shuffle(only_train=False)

# shuffle the clusters within each event
if shuffle_clusters:
    ai.data.shuffle_training_clusters()

In [4]:
# define and create the neural network architecture
ai.create_model(conv_layers=[128, 64], classifier_layers=[32], type_layers=[8], 
                pos_layers=[64,32], energy_layers=[32, 16], base_l2=.0001, limbs_l2=.0001)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 72, 1)]      0                                            
__________________________________________________________________________________________________
conv_1 (Conv1D)                 (None, 8, 128)       1280        inputs[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv1D)                 (None, 8, 64)        8256        conv_1[0][0]                     
__________________________________________________________________________________________________
flatting (Flatten)              (None, 512)          0           conv_2[0][0]                     
______________________________________________________________________________________________

In [5]:
# compile the ai
ai.compile_model(learning_rate=0.0003)

# define the learning rate scheduler for the training phase
def lr_scheduler(epoch):
    if epoch < 90:
        return .0003
    elif epoch < 160:
        return .0001
    elif epoch < 180:
        return .00003
    elif epoch < 190:
        return .00001
    else:
        return .000003
l_callbacks = [
    keras.callbacks.LearningRateScheduler(lr_scheduler)
]

In [6]:
#TRAINING

In [7]:
%%time
# start the training
ai.train(epochs=200, shuffle=True, shuffle_clusters=shuffle_clusters, verbose=0, callbacks = l_callbacks)

/.automount/home/home__home2/institut_3b/profe/Software/DeepLearning/neural-network/sificc_lib/AI.py:25: RuntimeWarning: invalid value encountered in true_divide
  logs['pur'] = np.sum(l_matches) / np.sum(y_pred[:,0])
/.automount/home/home__home2/institut_3b/profe/Software/DeepLearning/neural-network/sificc_lib/AI.py:31: RuntimeWarning: invalid value encountered in true_divide
  logs['val_pur'] = np.sum(l_matches) / np.sum(y_pred[:,0])


CPU times: user 8min 40s, sys: 24min 5s, total: 32min 46s
Wall time: 4min 33s


In [17]:
# evaluate the AI on the training set
ai.model.evaluate(ai.data.train_x, ai.data.train_y, verbose=1)
print()

84/84 [==============================] - 2s 26ms/step - loss: 5.8556 - e_cluster_loss: 0.0450 - p_cluster_loss: 0.1259 - type_loss: 0.3519 - pos_x_loss: 0.4482 - pos_y_loss: 1.6709 - pos_z_loss: 0.2008 - energy_loss: 1.1625 - e_cluster__cluster_accuracy: 0.9363 - p_cluster__cluster_accuracy: 0.8154 - type__type_accuracy: 0.8419 - type__type_tp_rate: 0.7570



In [18]:
ai.evaluate()

AI model
  Loss:        6.97984
    -Type:         0.39359 *  2.00 = 0.78718
    -Pos X:        0.64156 *  2.50 = 1.60391
    -Pos Y:        1.64068 *  1.00 = 1.64068
    -Pos Z:        0.31045 *  2.00 = 0.62089
    -Energy:       1.38586 *  1.50 = 2.07880
    -Cls e:        0.08077 *  1.00 = 0.08077
    -Cls p:        0.12351 *  1.00 = 0.12351
  Accuracy:    0.82428
    -TP rate:      0.71765
    -Cls e rate:   0.92941
    -Cls p rate:   0.82353
  Efficiency:  0.00000
  Purity:      0.00000
  Euc mean:   13.87020
  Euc std:    10.12611

Reco
  Accuracy:    0.63898
    -TP rate:      0.71765
  Efficiency:  0.08235
  Purity:      0.04667
  Euc mean:   17.04447
  Euc std:    44.69097


In [10]:
# save the trained model
#ai.save(file_name=model_name)

In [11]:
# Load the trained model
#ai.load(file_name=model_name)